In [15]:
import torch 
import torch.nn as nn
import torchmetrics
from torch.utils.data import Dataset
import pandas as pd

In [16]:
import torch.nn as nn 
import numpy as np

torch.manual_seed(42)

np.random.seed(42)

BATCH_SIZE = 32
LR = 0.001
CLASSES = 6

TRAIN_VAL_TEST_SPLIT = [0.7, 0.15, 0.15]

EPOCHS = 40
LOSS_FUNCTION =nn.CrossEntropyLoss(weight=torch.tensor([2.0, 1.0,3.0,4.0,8.0,4.0]))#pos_weight=torch.tensor([10.0])

AUGMENT = True
SAVE_BEST_MODEL = True
IS_MULTICLASS = True if CLASSES > 2 else False
NUM_OF_WORKERS = 0

REDUCE_LR_PATIENCE = 10
EARLY_STOPPING_PATIENCE = 10

In [17]:
'''

wandb.init(project="zneus-project-1-binary", name="Basic model")

# Config hyperparameters
config = wandb.config
config.batch_size = BATCH_SIZE
config.learning_rate = LR
config.epochs = EPOCHS
config.desc = ""
config.loss_function = "Crossentropy "
config.reduce_lr_patience = REDUCE_LR_PATIENCE
config.early_stopping_patience = EARLY_STOPPING_PATIENCE
'''

'\n\nwandb.init(project="zneus-project-1-binary", name="Basic model")\n\n# Config hyperparameters\nconfig = wandb.config\nconfig.batch_size = BATCH_SIZE\nconfig.learning_rate = LR\nconfig.epochs = EPOCHS\nconfig.desc = ""\nconfig.loss_function = "Crossentropy "\nconfig.reduce_lr_patience = REDUCE_LR_PATIENCE\nconfig.early_stopping_patience = EARLY_STOPPING_PATIENCE\n'

In [18]:
[i for i in range(6)], [*"Pastry Z_Scratch K_Scratch Stains Dirtiness Bumps".split(" ")] # V28 V29 V30 V31 V32 V33

([0, 1, 2, 3, 4, 5],
 ['Pastry', 'Z_Scratch', 'K_Scratch', 'Stains', 'Dirtiness', 'Bumps'])

In [19]:

class SteelPlateDataset(Dataset):

    def __init__(self, dataset_path):
        super().__init__()
        self.path = dataset_path
        self.df = pd.read_csv(self.path)

        self.features = self.df.drop(["Class", *("Pastry Z_Scratch K_Scratch Stains Dirtiness Bumps".split(" "))] ,axis= 1).values.tolist() # V28 V29 V30 V31 V32 V33
        self.labels =  self.df[[*"Pastry Z_Scratch K_Scratch Stains Dirtiness Bumps".split(" ")]].dot([i for i in range(6)]).values.tolist() # V28 V29 V30 V31 V32 V33
       
    def __getitem__(self, index):

        return torch.tensor(self.features[index]), torch.tensor(self.labels[index]).float()

    def __len__(self):
        return len(self.labels)
        


In [20]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset

dataset = SteelPlateDataset("data/norm_data.csv")

labels = np.array(dataset.labels)
print(labels)
train_size, val_size, test_size = TRAIN_VAL_TEST_SPLIT

train_val_indices, test_indices = train_test_split(
    range(len(dataset)),
    test_size=TRAIN_VAL_TEST_SPLIT[1],
    stratify=dataset.labels,
    random_state=42
)

# Update labels for train+val indices
train_val_labels = labels[train_val_indices]

# Second split: train vs val
train_indices, val_indices = train_test_split(
    train_val_indices,
    test_size=val_size / (train_size + val_size),  # Adjust proportion for train+val
    stratify=train_val_labels,
    random_state=42
)



train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)


[-5.15991109 -5.15991109 -5.15991109 ... -5.15991109 -5.15991109
 -5.15991109]


In [21]:
len(dataset.features[0])

23

In [22]:
if torch.cuda.is_available():
    device_name = "cuda"
elif torch.backends.mps.is_available():
    device_name = "mps"
else:
    device_name = "cpu"

In [23]:
from torch.utils.data import DataLoader



train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    
    shuffle=True,  # Default shuffling for training
    num_workers=NUM_OF_WORKERS
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,  # No shuffling for validation
    num_workers=NUM_OF_WORKERS
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,  # No shuffling for test
    num_workers=NUM_OF_WORKERS
)

# Print dataset sizes
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 1358
Validation dataset size: 291
Test dataset size: 292


In [24]:
import torch

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):
        """
        Args:
            patience (int): How many epochs to wait after last improvement.
            min_delta (float): Minimum change to qualify as an improvement.
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0


In [25]:
from torchmetrics.classification import Accuracy, Precision
from torch.optim.lr_scheduler import ReduceLROnPlateau


class MyModel(nn.Module):
    def __init__(self, input_size,lr=0.001, loss_fn=nn.BCELoss(), num_classes=2,reduce_lr_patience=10, early_stopping_patience=10):
        super().__init__()
        self.accuracy = Accuracy(task="multiclass", num_classes=num_classes,average="macro")
        self.precision = Precision(task="multiclass", num_classes=num_classes, average="macro")
        self.loss_fn = loss_fn


        self.model = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, num_classes if num_classes > 2 else 1)
        )
        self.to(device_name)

        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)

        self.early_stopping = EarlyStopping(patience=early_stopping_patience, min_delta=0.01)

        self.scheduler = ReduceLROnPlateau(
            self.optimizer, mode='min', factor=0.5, patience=reduce_lr_patience
        )

    def forward(self, x):
        return self.model(x)
        
        
    def evaluate(self, dataloader):
        self.eval()
        eval_loss = 0

        self.precision.reset()
        self.accuracy.reset()

        

        with torch.no_grad():
            for batch in dataloader:

                x = batch[0].to(device_name)
                y = batch[1].to(device_name)

                output = self.forward(x)
                output = torch.sigmoid(output).squeeze(1)
                y = y.float()

                
                    
                loss = self.loss_fn(output, y)
 
                self.accuracy(output, y)
                self.precision(output, y)
              
          
                eval_loss += loss.item()

        self.train()
        return (eval_loss/len(dataloader), self.accuracy.compute(), self.precision.compute())
    
        
    def fit(self, train_dataloader, val_dataloader, epochs=10):
        self.train()
        best_val_loss = 9999

     
      
        for i in range(0,epochs):
           
            self.accuracy.reset()
            epoch_loss = 0
            for batch in train_dataloader:

                x = batch[0].to(device_name)
                y = batch[1].to(device_name)
              

                output = self.forward(x)
                output = torch.sigmoid(output).squeeze(1)
                y = y.float()
                
            
                
                loss = self.loss_fn(output, y)

               
                self.accuracy(output, y)
                self.precision(output, y)

                epoch_loss += loss.item()

                self.zero_grad()
                loss.backward()
                self.optimizer.step()


            epoch_loss/=len(train_dataloader)
           
            epoch_acc = self.accuracy.compute()
            epoch_precision = self.precision.compute()

       

            val_loss, val_acc, val_precision = self.evaluate(val_dataloader)
            if best_val_loss > val_loss:
                best_val_loss = val_loss
                torch.save(self.state_dict(), "best-model-multiclass.pth")

            self.early_stopping(val_loss)
            if self.early_stopping.early_stop:
                print("Early stopping")
                break

            self.scheduler.step(val_loss)
            #wandb.log({"epoch": i, "Train Loss": epoch_loss, "Train Acc":epoch_acc,"Train F1":epoch_f1, "Val Loss":val_loss, "Val Acc":val_acc,"Val F1":val_f1, "LR":self.optimizer.param_groups[0]['lr']})
            print(f"Epoch {i+1} Loss:{epoch_loss:.4f} Accuracy:{epoch_acc:.4f} Precision:{epoch_precision:.4f} Val Loss:{val_loss:.4f} Val Accuracy:{val_acc:.4f} Val Precision:{val_precision:.4f} LR = {self.optimizer.param_groups[0]['lr']}")
        #wandb.finish()
        


In [26]:
model = MyModel(
    input_size=len(dataset.features[0]),
    num_classes=CLASSES, 
    loss_fn=LOSS_FUNCTION, lr=LR, 
    reduce_lr_patience=REDUCE_LR_PATIENCE, 
    early_stopping_patience=EARLY_STOPPING_PATIENCE
)

In [27]:
model.fit(train_dataloader, val_dataloader, epochs=EPOCHS)

Epoch 1 Loss:1.1044 Accuracy:0.2454 Precision:0.2500 Val Loss:1.0459 Val Accuracy:1.0000 Val Precision:1.0000 LR = 0.001
Epoch 2 Loss:1.0518 Accuracy:0.4982 Precision:0.5000 Val Loss:1.0445 Val Accuracy:1.0000 Val Precision:1.0000 LR = 0.001
Epoch 3 Loss:1.0472 Accuracy:1.0000 Precision:1.0000 Val Loss:1.0450 Val Accuracy:1.0000 Val Precision:1.0000 LR = 0.001
Epoch 4 Loss:1.0448 Accuracy:1.0000 Precision:1.0000 Val Loss:1.0442 Val Accuracy:1.0000 Val Precision:1.0000 LR = 0.001
Epoch 5 Loss:1.0445 Accuracy:1.0000 Precision:1.0000 Val Loss:1.0440 Val Accuracy:1.0000 Val Precision:1.0000 LR = 0.001
Epoch 6 Loss:1.0442 Accuracy:1.0000 Precision:1.0000 Val Loss:1.0438 Val Accuracy:1.0000 Val Precision:1.0000 LR = 0.001
Epoch 7 Loss:nan Accuracy:1.0000 Precision:1.0000 Val Loss:1.0438 Val Accuracy:1.0000 Val Precision:1.0000 LR = 0.001
Epoch 8 Loss:1.0439 Accuracy:1.0000 Precision:1.0000 Val Loss:1.0437 Val Accuracy:1.0000 Val Precision:1.0000 LR = 0.001
Epoch 9 Loss:1.0438 Accuracy:1.0000

In [28]:
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Example data (each pixel has an integer class label)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=len(val_dataset),
    shuffle=False,  # Use sampler instead of shuffle
    num_workers=NUM_OF_WORKERS
)

features, ground_truth = next(iter(val_dataloader))

images = features.to(device_name)
model.eval()
predictions = model(images)
print(ground_truth)
predictions =  torch.argmax(predictions, dim=1).float()
y_true = ground_truth.numpy()
y_pred = predictions.detach().cpu().int().numpy()

y_true_flat = y_true.flatten()
y_pred_flat = y_pred.flatten()

cm = confusion_matrix(y_true_flat, y_pred_flat, labels=[0, 1, 2,3,4,5])
print(cm)
# Convert to percentage (%)
cm_percent = cm.astype('float') / cm.sum(axis=1, keepdims=True) * 100
disp = ConfusionMatrixDisplay(confusion_matrix=cm_percent,
                          display_labels=["Pastry", "Z_Scratch","K_Scratch"	,"Stains"	,"Dirtiness"	,"Bumps"])

disp.plot(cmap='Blues', values_format='.1f')



tensor([ 7.1754, -5.1599, -5.1599,  7.1754, -0.1746, -5.1599, -5.1599,  7.1754,
         7.1754,  7.1754, -5.1599, -0.1746, -5.1599, -5.1599, -0.1746, -5.1599,
        -5.1599, -5.1599, -5.1599, -5.1599, -5.1599, -1.7956, -0.1746, -5.1599,
        -5.1599,  7.1754, -0.1746, 18.9403, -5.1599,  7.1754, -5.1599, -0.1746,
        -5.1599,  7.1754, -5.1599, -5.1599, -1.7956,  7.1754, -0.1746,  7.1754,
        -0.1746, -0.1746, -0.1746,  7.1754, -5.1599, -5.1599, -5.1599, -5.1599,
        -1.7956, -1.7956, -0.1746, -1.7956, -1.7956, -5.1599, -0.1746,  7.1754,
        -0.1746, -5.1599, 10.7096, -5.1599,  7.1754, 18.9403,  7.1754,  7.1754,
         7.1754, -5.1599,  7.1754, -5.1599,  7.1754, -5.1599, 10.7096, -5.1599,
        -5.1599, -1.7956, -1.7956, -5.1599, -5.1599,  7.1754,  7.1754, -1.7956,
        -1.7956, -0.1746, -5.1599, -5.1599,  7.1754,  7.1754, -5.1599, -5.1599,
        -5.1599, -5.1599,  7.1754, -5.1599, -5.1599, -5.1599, -0.1746, -0.1746,
        -1.7956,  7.1754, 10.7096, 18.94

ValueError: Classification metrics can't handle a mix of continuous and multiclass targets

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Example data (each pixel has an integer class label)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=len(val_dataset),
    shuffle=False,  # Use sampler instead of shuffle
    num_workers=NUM_OF_WORKERS
)

features, ground_truth = next(iter(val_dataloader))

images = features.to(device_name)
model.eval()
predictions = model(images)
print(ground_truth)
predictions = torch.argmax(predictions, dim=1).int()


y_true = ground_truth.numpy()
print(y_true)
y_pred = predictions.detach().cpu().int().numpy()

y_true_flat = y_true.flatten()
y_pred_flat = y_pred.flatten()


cm = confusion_matrix(y_true_flat, y_pred_flat, labels=[0, 1, 2,3,4,5])
print(cm)
# Convert to percentage (%)
cm_percent = cm.astype('float') / cm.sum(axis=1, keepdims=True) * 100
disp = ConfusionMatrixDisplay(confusion_matrix=cm_percent,
                          display_labels=["Pastry", "Z_Scratch","K_Scratch"	,"Stains"	,"Dirtiness"	,"Bumps"])

disp.plot(cmap='Blues', values_format='.1f')



   




In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Example data (each pixel has an integer class label)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=len(test_dataset),
    shuffle=False,  # Use sampler instead of shuffle
    num_workers=NUM_OF_WORKERS
)

features, ground_truth = next(iter(test_dataloader))

images = features.to(device_name)
model.eval()
predictions = model(images)
print(predictions.shape)
predictions =  torch.argmax(predictions, dim=1).float()
y_true = ground_truth.numpy()
y_pred = predictions.detach().cpu().int().numpy()

y_true_flat = y_true.flatten()
y_pred_flat = y_pred.flatten()

cm = confusion_matrix(y_true_flat, y_pred_flat, labels=[0, 1, 2,3,4,5])
print(cm)
# Convert to percentage (%)
cm_percent = cm.astype('float') / cm.sum(axis=1, keepdims=True) * 100
disp = ConfusionMatrixDisplay(confusion_matrix=cm_percent,
                          display_labels=["Pastry", "Z_Scratch",	"K_Scratch"	,"Stains"	,"Dirtiness"	,"Bumps"])

disp.plot(cmap='Blues', values_format='.1f')



   




In [ ]:
cm = confusion_matrix(y_true_flat, y_pred_flat, labels=[0, 1, 2,3,4,5])
print(cm)
# Convert to percentage (%)
cm_percent = cm.astype('float') / cm.sum(axis=1, keepdims=True) * 100
disp = ConfusionMatrixDisplay(confusion_matrix=cm_percent,
                          display_labels=["Pastry", "Z_Scratch",	"K_Scratch"	,"Stains"	,"Dirtiness"	,"Bumps"])

disp.plot(cmap='Blues', values_format='.1f')